In [1]:
import pandas as pd
import numpy as np
import statsmodels as stt
import scipy.stats as sst
import os.path as osp
from statsmodels import api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [2]:
print(osp.realpath(osp.curdir))
relative_path_filename = './data/data.csv'
assert osp.exists(relative_path_filename)

/home/jb/code/repronim/simple2/simple2_analysis


In [3]:
hie = pd.read_csv(relative_path_filename, na_values='nd') #, low_memory=False)
original_col_names = list(hie)
# column names are unique
assert len(original_col_names) == len(set(original_col_names))
print(list(hie))
col_rename = {'federatedLabel':'structure'}
hie.rename(columns=col_rename, inplace=True)
print(list(hie))

['rowid', 'study', 'ID', 'Age', 'dx', 'Gender', 'FIQ', 'PIQ', 'VIQ', 'tool', 'softwareLabel', 'federatedLabel', 'laterality', 'volume']
['rowid', 'study', 'ID', 'Age', 'dx', 'Gender', 'FIQ', 'PIQ', 'VIQ', 'tool', 'softwareLabel', 'structure', 'laterality', 'volume']


/home/jb/virtualenvs/simple2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# mapping_file = '../segstats_jsonld/segstats_jsonld/mapping_data/freesurfermap.json'
mapping_file = '../segstats_jsonld/segstats_jsonld/mapping_data/freesurfer-cdes.json'
assert osp.exists(mapping_file)
with open(mapping_file, "r") as read_file:
    roi_map = json.load(read_file)

In [5]:
ube2h = {}
label2ube = {}
countok=0
has_no_isAbout = []
has_no_label = []

for (k,v) in roi_map.items():
    
    # discard 'count'
    if k == 'count': pass
    
    # v is a dict that contains the CDE - check that we have a isAbout and label    
    elif 'isAbout' in v:
        countok += 1
        if 'label' in v:
            if v['label'] != '' and v['label'] not in ('None','none'):
                #ube['<' + v['isAbout'] + '>'] = v['label']
                #ebu[v['label']] = '<' + v['isAbout'] + '>'
                label2ube[v['label']] = v['isAbout']
                if v['isAbout'] not in ube2h.keys():
                    no_right_or_left = v['label']
                    no_right_or_left = no_right_or_left.replace('Right-','')
                    no_right_or_left = no_right_or_left.replace('Right ','')
                    no_right_or_left = no_right_or_left.replace('Left-','')
                    no_right_or_left = no_right_or_left.replace('Left ','')
                    no_right_or_left = no_right_or_left.replace(' NVoxels','')
                    no_right_or_left = no_right_or_left.replace(' (mm^3)','')
                    ube2h[v['isAbout']] = no_right_or_left
            else:
                has_no_label.append(k)

assert has_no_isAbout == []
assert countok == len(label2ube)

In [6]:
label2ube;
ube2h
h2ube = {v: k for k, v in ube2h.items()}

In [56]:
def split_merge_df(df, indx='ID', spliton='laterality', levels=['Left','Right'], 
                       keep_col='volume', op='+', colrename=None):
    """
    1. split the df according to 2 (n?) levels of "spliton"
    2. merge the 2 (n?) df using indx as index
    3. keep only "keep_col" for the right temporary df
    4. perform op on the columns "keep_col" and name it 'keep_col'+'_'+ levels[0] + op + levels[1]    
    
    example: for adding volumes in right and left structures
    """
    
    dflev1 = df[df[spliton]==levels[0]]
    dflev2 = df[df[spliton]==levels[1]] 

    # check that the new dfs have no duplicates in the indx

    assert set(dflev1[indx]) == set(dflev2[indx])
    assert len(set(dflev1[indx])) == len(dflev1[indx])
    
    # assert len(set(dflev2[indx])) == len(dflev2[indx])
    # suffixes=('_l','_r')
    merged_inner = pd.merge(left=dflev1, right=dflev2[[indx,keep_col]], 
                            left_on=indx, right_on=indx, suffixes=levels, how='inner')
#    merged_inner.rename(columns={cols+'_x': cols+'_'+lev1, cols+'_y': cols+'_'+lev2}, inplace=True)

    # sum keep_col values in a new column
    add_col_name = keep_col + levels[0] + op + levels[1]
    if op == '+':
        merged_inner[add_col_name] = \
                        merged_inner[keep_col+levels[0]] + merged_inner[keep_col+levels[1]]  
    if colrename is not None:
        merged_inner.rename(columns={add_col_name:colrename}, inplace=True)
    return merged_inner

"""
    if droplist != []:
        for colname in droplist:
            colname_y = colname + '_y'
            colname_x = colname + '_x'
            merged_inner.drop(colname_y, axis=1, inplace=True)
            merged_inner.rename(columns={colname_x: colname}, inplace=True)
""";

In [86]:
tooldic = {'surfer':'https://surfer.nmr.mgh.harvard.edu/', 
       'fsl':'http://purl.org/nidash/fsl#',
       'ants':'http://stnava.github.io/ANTs/'}
normalDev = (2, '2', 'Typically Developing Children')
adhd = (1, '1', 'ADHD-Combined', 'ADHD-Hyperactive/Impulsive', 'ADHD-Inattentive')


def define_conditions(df, tooldic={}, normalDev=(), adhd=(), h2ube={}):
    """
    create dic of conditions
    
    """
    diccond={}
    diccond['left'] = ((df['laterality'] == 'L')|(hie['laterality'] == 'Left'))
    diccond['right'] = ((df['laterality'] == 'R')|(hie['laterality'] == 'Right'))
    diccond['age20'] = (df['Age'] <= 20)
    
    #========== tool conditions
    diccond['fs'] = (df['tool'] == tooldic['surfer'])
    diccond['ants'] = (df['tool'] == tooldic['ants'])
    diccond['fsl'] = (df['tool'] == tooldic['fsl'])
    
    #========== IQ conditions
    diccond['fiq>0'] = (df['FIQ'] > 0 )
    
    #========== disease conditions
    pop_normDev = False
    for pop in normalDev:
        # print(np.sum(pop_cond))
        pop_normDev = pop_normDev | (df['dx'] == pop)
    diccond['normDev'] =  pop_normDev
    
    pop_adhd = False
    for pop in adhd:
        # print(np.sum(pop_cond))
        pop_adhd = pop_adhd | (df['dx'] == pop)
    diccond['adhd'] =  pop_adhd

    #=========== ROIs
    roi_ub = ''
    diccond['bvol'] = (df['softwareLabel'] == 'BVOL (mm^3)')
    diccond['brainseg'] = (df['softwareLabel'] == 'Brain Segmentation Volume (mm^3)') 
    diccond['caudate'] = (df['structure'] == h2ube['Caudate'])
    diccond['putamen'] = (df['structure'] == h2ube['Putamen'])
    diccond['TIV'] = (df['structure'] == h2ube['Estimated Total Intracranial Volume'])
    diccond['wm'] =  (df['structure'] == h2ube['hemisphere cerebral white matter volume'])
    diccond['gm'] =  (df['structure'] == h2ube['Total gray matter volume'])
    diccond['csf'] =  (df['structure'] == h2ube['CSF'])
    diccond['ccant'] =  (df['structure'] == h2ube['CC_Anterior'])
    diccond['cccen'] =  (df['structure'] == h2ube['CC_Central'])
    diccond['ccpos'] =  (df['structure'] == h2ube['CC_Posterior'])

    #=========== site
    diccond['abide'] = (df['study'].str.contains("ABIDE"))
    diccond['adhd200'] = (df['study'] == 'ADHD200')
    
    #=========== Gender
    diccond['male'] = (df['Gender']=='Male')
    diccond['female'] = (df['Gender']=='Female')

    return diccond


In [43]:
condic = define_conditions(hie, tooldic=tooldic, normalDev=normalDev, adhd=adhd, h2ube=h2ube)
condic.keys()

dict_keys(['left', 'right', 'age20', 'fs', 'ants', 'fsl', 'fiq>0', 'normDev', 'adhd', 'bvol', 'brainseg', 'caudate', 'putamen', 'TIV', 'wm', 'gm', 'csf', 'abide', 'adhd200', 'male', 'female'])

In [44]:
def apply_cond(df, cndc, conditions, dropnaset=[]):
    """
    List of conditions
    """
    cond = np.full((len(df),), True, dtype=bool)
    
    for c in conditions:
        cond = cond & cndc[c]
        # print(np.sum(cond))
        
    # condition = [cond & cndc[c] for c in conditions][0]
    # print(len(condition),np.sum(condition))
    
    # make a copy
    tmp = df.loc[cond].dropna(subset=dropnaset)
    return tmp
    

In [45]:
# hyp1 = ['bvol','ants','left','age20','fiq>0','normDev']
# hyp1 = ['fiq>0','abide','caudate', 'putamen']
hyp1 = ['female', 'caudate','fs','fiq>0'] # ,'abide']
condic = define_conditions(hie, tooldic=tooldic, normalDev=normalDev, adhd=adhd, h2ube=h2ube)
tmp = apply_cond(hie, condic, hyp1)
# print('apply:', len(tmp.dropna(subset=['FIQ', 'volume', 'Gender'])))

manual = hie.loc[(hie['Gender']=='Female') & 
#                 (hie['study'].str.contains("ABIDE")) &
                 (hie['structure'] == h2ube['Caudate']) &
                 (hie['tool'] == tooldic['surfer']) & 
                 (hie['FIQ'] > 0) ] 
manual = manual.dropna(subset=['FIQ', 'volume', 'Gender']) #,inplace=True)
print('manual: ',len(manual))
assert len(tmp.dropna(subset=['FIQ', 'volume', 'Gender'])) == len(manual)


manual:  422


In [46]:
structures = set(hie.structure)


Hypotheses

PIET-1: Total Brain Volume will positively correlate with IQ (in both sexes across the complete age range).

MAC-1: Left striatum volume (caudate + putamen) will positively correlate with IQ in the total (male + female) child (age < 20) group.

MAC-2: Left striatum volume (caudate + putamen) will positively correlate with IQ in the male children group.

MAC-3: Left striatum volume (caudate + putamen) will not correlate with IQ in the female children group.

GANJ-1: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will negatively correlate with IQ.

GANJ-2: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will negatively correlate with IQ in the young (age < 12) group.

GANJ-3: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will not significantly correlate with IQ in the adolescent (age > 12) group.

GANJ-4:. Total Corpus Callosum midsagittal area, after correcting for total brain volume, will negatively correlate with IQ in the male (age < 12) group.

GANJ-5: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will not significantly correlate with IQ in the female (age < 12) group.


### PIET-1: Total Brain Volume will positively correlate with IQ (in both sexes across the complete age range).


In [47]:
condic = define_conditions(hie, tooldic=tooldic, normalDev=normalDev, adhd=adhd, h2ube=h2ube)
print(condic.keys())
dropnaset = ['FIQ', 'volume', 'Gender']


dict_keys(['left', 'right', 'age20', 'fs', 'ants', 'fsl', 'fiq>0', 'normDev', 'adhd', 'bvol', 'brainseg', 'caudate', 'putamen', 'TIV', 'wm', 'gm', 'csf', 'abide', 'adhd200', 'male', 'female'])


In [48]:
hyp1 = ['bvol', 'fiq>0','normDev','ants'] # ,'abide']
tmp = apply_cond(hie, condic, hyp1, dropnaset=dropnaset)
print(len(tmp))

458


In [49]:
print(" Structure = ", 'bvol')
#assert ube2h[tmp.iloc[0]['structure']] == roi

iq = 'FIQ'

# md = smf.ols(iq + " ~ Q('volume') + Gender + Age + study ", data=tmp) #  
md = smf.ols(iq + " ~ Q('volume') + study ", data=tmp) #  
mdf = md.fit()
print(mdf.summary())

 Structure =  bvol
                            OLS Regression Results                            
Dep. Variable:                    FIQ   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     3.583
Date:                Mon, 02 Dec 2019   Prob (F-statistic):           1.28e-06
Time:                        07:03:17   Log-Likelihood:                -1791.6
No. Observations:                 458   AIC:                             3621.
Df Residuals:                     439   BIC:                             3700.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

### MAC-1: Left striatum volume (caudate + putamen) will positively correlate with IQ in the total (male + female) child (age < 20) group.


In [77]:
hyp2_caud = ['fiq>0','normDev','fs','age20','caudate'] # ,'abide']
hyp2_put = ['fiq>0','normDev','fs','age20','putamen'] # ,'abide']
hyp2_tiv = ['fiq>0','normDev','fs','age20','TIV'] # ,'abide']
tmp_caud = apply_cond(hie, condic, hyp2_caud, dropnaset=dropnaset)
caud = split_merge_df(tmp_caud, indx='ID', spliton='laterality', levels=['Left','Right'], 
                       keep_col='volume', op='+',colrename='caudate')
tmp_put = apply_cond(hie, condic, hyp2_put, dropnaset=dropnaset)
put = split_merge_df(tmp_put, indx='ID', spliton='laterality', levels=['Left','Right'], 
                       keep_col='volume', op='+',colrename='putamen')
tmp_tiv = apply_cond(hie, condic, hyp2_tiv, dropnaset=dropnaset)
print(len(caud), len(put), len(tmp_tiv))

stria = pd.merge(left=caud, right=put[['ID','putamen']], left_on='ID', right_on='ID')
stria['striatum'] = stria['caudate']+stria['putamen']
stria = pd.merge(left=stria, right=tmp_tiv[['ID','volume']], left_on='ID', right_on='ID')
stria.rename(columns={'volume':'TIV'},inplace=True)
print(list(stria),len(stria))

409 409 409
['rowid', 'study', 'ID', 'Age', 'dx', 'Gender', 'FIQ', 'PIQ', 'VIQ', 'tool', 'softwareLabel', 'structure', 'laterality', 'volumeLeft', 'volumeRight', 'caudate', 'putamen', 'striatum', 'TIV'] 409


In [78]:
print(" Structure = ", 'striatum')

iq = 'FIQ'

# md = smf.ols(iq + " ~ Q('volume') + Gender + Age + study ", data=tmp) #  
# md = smf.ols(iq + " ~ Q('striatum') + study + TIV ", data=stria) #  
md = smf.ols(iq + " ~ Q('striatum') + study ", data=stria) #  
mdf = md.fit()
print(mdf.summary())

 Structure =  striatum
                            OLS Regression Results                            
Dep. Variable:                    FIQ   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     2.631
Date:                Mon, 02 Dec 2019   Prob (F-statistic):           0.000340
Time:                        07:27:44   Log-Likelihood:                -1587.1
No. Observations:                 409   AIC:                             3212.
Df Residuals:                     390   BIC:                             3288.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

### MAC-2: Left striatum volume (caudate + putamen) will positively correlate with IQ in the male children group.


In [80]:
hyp3_caud = ['fiq>0','normDev','fs','age20','caudate','left','male'] # ,'abide']
hyp3_put = ['fiq>0','normDev','fs','age20','putamen','left','male'] # ,'abide']
hyp3_tiv = ['fiq>0','normDev','fs','age20','TIV','male'] # ,'abide']
caud = apply_cond(hie, condic, hyp3_caud, dropnaset=dropnaset)
caud.rename(columns={'volume':'caudate'},inplace=True)

put = apply_cond(hie, condic, hyp3_put, dropnaset=dropnaset)
put.rename(columns={'volume':'putamen'},inplace=True)

tiv = apply_cond(hie, condic, hyp3_tiv, dropnaset=dropnaset)
print(len(caud), len(put), len(tiv))

stria = pd.merge(left=caud, right=put[['ID','putamen']], left_on='ID', right_on='ID')
stria['striatum'] = stria['caudate']+stria['putamen']
stria = pd.merge(left=stria, right=tmp_tiv[['ID','volume']], left_on='ID', right_on='ID')
stria.rename(columns={'volume':'TIV'},inplace=True)
print(list(stria),len(stria))

321 321 321
['rowid', 'study', 'ID', 'Age', 'dx', 'Gender', 'FIQ', 'PIQ', 'VIQ', 'tool', 'softwareLabel', 'structure', 'laterality', 'caudate', 'putamen', 'striatum', 'TIV'] 321


In [83]:
print(" Structure = ", 'striatum')

iq = 'FIQ'

# md = smf.ols(iq + " ~ Q('volume') + Gender + Age + study ", data=tmp) #  
#md = smf.ols(iq + " ~ Q('striatum') + study + TIV ", data=stria) #  
md = smf.ols(iq + " ~ Q('striatum') + study ", data=stria) #  
mdf = md.fit()
print(mdf.summary())

 Structure =  striatum
                            OLS Regression Results                            
Dep. Variable:                    FIQ   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     2.818
Date:                Mon, 02 Dec 2019   Prob (F-statistic):           0.000280
Time:                        07:36:25   Log-Likelihood:                -1231.4
No. Observations:                 321   AIC:                             2497.
Df Residuals:                     304   BIC:                             2561.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

#### hyp3 note : only correlates when TIV is not covaried out

### MAC-3: Left striatum volume (caudate + putamen) will not correlate with IQ in the female children group.


In [84]:
hyp4_caud = ['fiq>0','normDev','fs','age20','caudate','left','female'] # ,'abide']
hyp4_put = ['fiq>0','normDev','fs','age20','putamen','left','female'] # ,'abide']
hyp4_tiv = ['fiq>0','normDev','fs','age20','TIV','female'] # ,'abide']
caud = apply_cond(hie, condic, hyp4_caud, dropnaset=dropnaset)
caud.rename(columns={'volume':'caudate'},inplace=True)

put = apply_cond(hie, condic, hyp4_put, dropnaset=dropnaset)
put.rename(columns={'volume':'putamen'},inplace=True)

tiv = apply_cond(hie, condic, hyp4_tiv, dropnaset=dropnaset)
print(len(caud), len(put), len(tiv))

stria = pd.merge(left=caud, right=put[['ID','putamen']], left_on='ID', right_on='ID')
stria['striatum'] = stria['caudate']+stria['putamen']
stria = pd.merge(left=stria, right=tmp_tiv[['ID','volume']], left_on='ID', right_on='ID')
stria.rename(columns={'volume':'TIV'},inplace=True)
print(list(stria),len(stria))

88 88 88
['rowid', 'study', 'ID', 'Age', 'dx', 'Gender', 'FIQ', 'PIQ', 'VIQ', 'tool', 'softwareLabel', 'structure', 'laterality', 'caudate', 'putamen', 'striatum', 'TIV'] 88


In [85]:
print(" Structure = ", 'striatum')

iq = 'FIQ'
#md = smf.ols(iq + " ~ Q('striatum') + study + TIV ", data=stria) #  
md = smf.ols(iq + " ~ Q('striatum') + study ", data=stria) #  
mdf = md.fit()
print(mdf.summary())

 Structure =  striatum
                            OLS Regression Results                            
Dep. Variable:                    FIQ   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.9409
Date:                Mon, 02 Dec 2019   Prob (F-statistic):              0.521
Time:                        11:56:31   Log-Likelihood:                -348.27
No. Observations:                  88   AIC:                             726.5
Df Residuals:                      73   BIC:                             763.7
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

### GANJ-1: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will negatively correlate with IQ.
